# RSA Cryptography explanation

RSA cryptography remains a mathematically secure method of performing asymmetric cryptography.  In this notebook, we will provide an example and explain the mathematical fundamentals of the encryption decryption process.

## Background

Data sent through the internet will be go through various nodes, i.e. (switches, routers and hubs) before reaching the final destination.  It must be assumed that at every node, there exists an opportunity for an eavesdropper to monitor and log all data that goes through.  Thus, data encryption is necessary to prevent the interception of sensitive data in this manner; a sender and receiver both possess a key that will encrypt data when sent through the internet and decrypt data after it has reached its destination.

This introduces another logistical issue in that sending an encryption key sent through the internet (to initiate the exchange of encrypted data) will also be intercepted by eavesdroppers thus defeating the purpose of any further encryption.  To counter this, is necessary for the encryption key to be different than the decryption key.  I.E. If Bob wants to initiate a secure communication channel through the internet, Bob would broadcast an encryption key that everyone can use to encrypt data sent to him, but only Bob will have the decryption key to decrypt data sent to him.

## Mathematical basics

RSA cryptography requires the use of prime numbers with some specific conditions.  For that we will introduce two functions; the Miller Rabin primality test and Euler's GCD theorem.


In [49]:
import random

def miller_rabin(n, k):

    if n == 2 or n == 3:
        return True

    if n % 2 == 0:
        return False

    r, s = 0, n - 1
    while s % 2 == 0:
        r += 1
        s //= 2
    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, s, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

def Eulers_GCD(a, b):
    r = a % b
    q = a//b
    while r != 0:
        a = b
        b = r
        q = a//b
        r = a - b*q
    return b

The Miller Rabin primality test will return true if an input number n has an overwhelmingly high probability of being prime, while Euler's GCD will return the greatest common denominator between two input numbers or simply 1 if there are none.

The next step is to find two prime numbers p and q.  In the line below, p is already set to an initial value, but you can set p to any number simply by modifying the line.

In [50]:
p = 6774

In the next block of code, p will be incremented until it passes the Miller Rabin primality test.

In [51]:
while miller_rabin(p, 40) == False:
    p += 1

Another prime number q is needed.  Similar to p before, q is given an initial value, but it can be modified to any other number.

In [52]:
q = 1445

Like before, q will be incremented until it passes the Miller Rabin primality test.

In [53]:
while miller_rabin(q, 40) == False:
    q += 1

print(p,q)

6779 1447


These are the two prime numbers we obtained.  

Using p and q, we acquire two other numbers of importance, the modulus:

In [54]:
modulus = p*q

and the totient

In [55]:
totient = (p-1)*(q-1)

print("The modulus and totient respectively:", modulus,totient)

The modulus and totient respectively: 9809213 9800988


Next, we need a prime number n that is coprime with the totient, i.e. the greatest common factor between them is 1.  Here we increment n until it passes the Miller Rabin test and has a value of 1 returned from the Euler's GCD function.  n should also be significantly smaller than the modulus

In [56]:
n = 65533

while miller_rabin(n, 40) == False or Eulers_GCD(totient, n) != 1:
    n += 1 

print(n)

65537


Now that we have our n, and totient, it is time to calculate the private key.
The private key (pk) is such that the below equation is satisfied.

n * pk mod totient = 1

where mod is the modulo function, or simply, when n * pk is divided by the totient, the remainder must be 1.  To accomplish this, we need will increment an integer i until the below equation

(i * totient + 1) / n yields an integer.  To accomplish this, we use the code below

In [57]:
def Get_Private_Key(totient, n):
    for i in range(0, n):
        if ((i * totient)+1) % n == 0:
            print("Found private key ")
            print("on iteration ", i)
            return ((i * totient)+1)//n
        
pkey = Get_Private_Key(totient, n)

Found private key 
on iteration  30556


Now we have all the numbers we need.  The modulus, and n is publicly broadcast as the public key, while the private key is kept secret.  To encrypt data, the formula is (plain text)^n mod modulus.

To efficiently take the modulus of a large exponential number, we use the following properties of modulo division as demonstrated in this example:

a mod b = c

a^2 mod b = c^2 mod b = d

a^4 mod b = d^2 mod b = e

a^8 mod b = e^2 mod b = f


To calculate the a^13 mod b is the same as (a^8 * a^2 * a^1)mod b

Which can be expressed as 
(a^8 mod b * a^2 mod b * a^1 mod b ) mod b

Or from using the coefficients in the table above
(c * d * f) mod b
